In [18]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [19]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re

In [23]:
# get doi from url
def url2doi(url):
    url = str(url).strip()

    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Firefox(options, service=Service(GeckoDriverManager().install()))
            driver.get(url)

            # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//p[text()='Consent']"))).click()

            # try:
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#formInput"))).send_keys(str(doi).strip())
            # except TimeoutException:
            #     print("Waiting for clicking consent timeout")
            # try:
            #     # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[2]/form/button"))).click()
            #     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[text()="Get PMID"]'))).click()
            # except TimeoutException:
            #     print("Waiting for clicking button timeout")
            
            error_label = 1
        except:
            print("URL to DOI transformation failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0

    try:
        my_elem = driver.find_elements(By.XPATH, "//*[contains(text(), 'doi.org') or contains(text(), 'DOI')]")
        doi = 
        doi = str(my_elem.find_element(By.XPATH, "i").get_attribute("innerHTML"))
    except:
        doi = np.nan
    driver.quit()
    return doi
# --------------------start of test code--------------------
url = "https://www.tandfonline.com/doi/abs/10.1080/01616412.1985.11739692"
doi = url2doi(url)
print(doi)
# ---------------------end of test code---------------------